In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/OML---mini-project/src

In [ ]:
!pip install -r requirements.txt

In [ ]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard library imports
import math
import os

# Third party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.optim.lr_scheduler as schedulers

# Local application imports
from utils import retrieve_setup, retrieve_training_params, training_loop

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "MobileNetV3Small"
dataset_name = "CIFAR10" # or "FashionMNIST"
n_epochs = 40
lr_max = 1e-3
lr_min = 1e-5

In [ ]:
# retrieves
model, dataset = retrieve_setup(model_name, dataset_name)
optimizer, _, loss_fn, _, batch_size = retrieve_training_params(model, dataset_name, "FixLR","parameters.yml")

# change lr from default to lr_max
for g in optimizer.param_groups:
    g["lr"] = lr_max

# implement linear increasing scheduler
total_iters = math.ceil(len(dataset["train"])/batch_size) * n_epochs
start_factor = lr_min/lr_max

scheduler_name, scheduler = "rangeLR", schedulers.LinearLR(
    optimizer,
    start_factor=start_factor,
    end_factor=1,
    total_iters=total_iters
)

In [ ]:
# run training loop
_, metrics = training_loop(
    model=model,
    dataset=dataset,
    scheduler=scheduler,
    optimizer=optimizer,
    loss_fn=loss_fn,
    n_epochs=n_epochs,
    batch_size=batch_size,
    train_strategy=("iter", 10),
    test_strategy=("iter", 10),
    scheduler_strategy="iter",
    file_name=f"/{dataset_name}_{scheduler_name}",
    device=device,
)

### Create plots (y-axis: accuracy, x-axis: learning rate)

In [ ]:
metric_folder = "metrics/lr_range_test"
file_names = ["CIFAR10_rangeLR.csv", "FashionMNIST_rangeLR.csv"]  # Add your file names here

plt.figure(figsize=(6, 4))

for file_name in file_names:
    df = pd.read_csv(os.path.join(metric_folder, file_name))

    lrs = df[df["source"] == "test"]["lr"].to_numpy()
    accuracy = df[df["source"] == "test"]["accuracy"].to_numpy()

    plt.plot(lrs, accuracy, label=file_name)

plt.legend()
plt.xlabel("Learning Rate")
plt.ylabel("Top-1 Accuracy")
plt.tight_layout()
plt.show()